In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score
from autogluon.tabular import TabularPredictor
import pickle
import os

In [ ]:
df_NACA4 = pd.read_csv("data/df_NACA4.csv")
df_NACA4.head(5)

In [ ]:
def split_data(df):
    df = df.sample(frac=1, random_state=0) 
    train = df.head(0)
    test = df.head(0)
    for ID in set(df['Airfoil_ID']):
        df_tmp = df[df['Airfoil_ID'] == ID]
        N = int(len(df_tmp) * 0.9)
        train = pd.concat([train, df_tmp.iloc[:N]])
        test = pd.concat([test, df_tmp.iloc[N:]])
    return train.reset_index(drop=True), test.reset_index(drop=True)

use_cols = ['M', 'P', 'XX', 'Re', 'AOA']
train_df, test_df = split_data(df_NACA4)

train = train_df[use_cols + ['Drag']]
test  = test_df[use_cols + ['Drag']]

In [ ]:
predictor = TabularPredictor(
    label='Drag',
    problem_type='regression',
    eval_metric='r2',
).fit(
    train_data=train,
    test_data=test,
    presets='best_quality',
    time_limit=1000, 
    verbosity=2,
)
pred = predictor.predict(test)
leader = predictor.leaderboard(test)

In [ ]:
print("MAE =", mean_absolute_error(test['Drag'], pred))
print("R2  =", r2_score(test['Drag'], pred))

In [ ]:
model_path = os.path.join(predictor.path, 'models', 'WeightedEnsemble_L2', 'model.pkl')
with open(model_path, 'rb') as f:
    ensemble = pickle.load(f)

print("Base models =", ensemble.base_model_names)

weights = ensemble._get_model_weights()
print("Weights     =", weights)